In [ ]:
import re
from collections import Counter
import os
from pathlib import Path
import datetime
from os import listdir
from os.path import isfile, join, isdir
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
import re
import multiprocessing
import pandas

In [44]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline

In [45]:
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})

In [104]:
%run ../_Util.ipynb

In [84]:
pastaArquivoCorpus = join('..', '..', 'resources', 'Corpora', '3- Textos Preprocessados', '_Consolidados')

In [86]:
list_stopwords = ['NUMBER','number','<NUMBER>', 'cid', 'cid0', 'cid1', 'cid2', 'cid3', 'cid32', 'et', 'al', 'petrobras1', 'figura', 'http', 'ser', 'pode', 'podem']

In [87]:
def printWordClouds(texto):
    wordcloud = WordCloud(
                          background_color='white',
                          stopwords=list_stopwords,
                          max_words=100,
                          max_font_size=50, 
                          #min_font_size=80,
                          random_state=42,
                          relative_scaling = 1, 
                          width = 800,
                          height = 600
                         ).generate(texto)
    
    print(wordcloud)
    fig = plt.figure(1)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()
    fig.savefig("wordcloud.png", dpi=900)

In [95]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items() if word not in list_stopwords]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]

#Most frequently occuring words
def printUnigramsHistogram(texto):
    corpus = [texto]
    #Convert most freq words to dataframe for plotting bar plot
    top_words = get_top_n_words(corpus, n=30)
    top_df = pandas.DataFrame(top_words)
    top_df.columns=["Word", "Freq"]
    pretty_print(top_df)
    
    #Barplot of most freq words
    g = sns.barplot(x="Word", y="Freq", data=top_df)
    g.set_xticklabels(g.get_xticklabels(), rotation=90)
    pretty_printCharts()

In [107]:
#Most frequently occuring Bi-grams
def get_top_n2_words(texto, n=None):
    corpus = [texto]
    vec1 = CountVectorizer(ngram_range=(2,2),  
            max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items() if word.split()[0] not in list_stopwords and word.split()[1] not in list_stopwords]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

def printBigramsHistogram(texto):
    corpus = texto
    
    top2_words = get_top_n2_words(corpus, n=30)
    top2_df = pandas.DataFrame(top2_words)
    top2_df.columns=["Bi-gram", "Freq"]
    pretty_print(top2_df)
    
    #Barplot of most freq Bi-grams
    import seaborn as sns
    #sns.set(rc={'figure.figsize':(13,8)})
    h=sns.barplot(x="Bi-gram", y="Freq", data=top2_df)
    h.set_xticklabels(h.get_xticklabels(), rotation=90)
    pretty_printCharts()

In [97]:
#Most frequently occuring Tri-grams
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3), 
           max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items() if word.split()[0] not in list_stopwords and word.split()[1] not in list_stopwords and word.split()[2] not in list_stopwords]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

def printTrigramsHistogram(texto):
    corpus = [texto]
    
    top3_words = get_top_n3_words(corpus, n=30)
    top3_df = pandas.DataFrame(top3_words)
    top3_df.columns=["Tri-gram", "Freq"]
    pretty_print(top3_df)
    
    #Barplot of most freq Tri-grams
    import seaborn as sns
    #sns.set(rc={'figure.figsize':(13,8)})
    j=sns.barplot(x="Tri-gram", y="Freq", data=top3_df)
    j.set_xticklabels(j.get_xticklabels(), rotation=90)
    pretty_printCharts()

In [106]:
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [ ]:
from IPython.display import display, HTML
def pretty_printCharts():
    CSS = """
    .output {
        flex-direction: row;
    }
    """

    HTML('<style>{}</style>'.format(CSS))